In [117]:
import torch 
from torch import nn, optim
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import StandardScaler

In [122]:
# Training features
X_train = pd.read_csv('../data/datasets/csv_files/Train_features.csv')
# Testing features
X_test = pd.read_csv('../data/datasets/csv_files/Test_features.csv')

In [121]:
# Training labels
y_train = pd.read_csv('../data/datasets/csv_files/Train_labels.csv')
# Testing labels
y_test = pd.read_csv('../data/datasets/csv_files/Test_labels.csv')

$$ \textbf{Préparation des données :} $$

$$ \text{Traitement des données d'entrainement} $$ 

In [120]:
import src.features.feature_selection as fsel

In [124]:
std = StandardScaler()

In [125]:
# Standardisation des ensembles d'entrainement et de test
training_feature_values = std.fit_transform(X_train)
testing_feature_values = std.fit_transform(X_test)

In [128]:
# Conversion des données d'entrainement en tensor 
X_train = torch.FloatTensor(training_feature_values)
y_train = torch.FloatTensor(y_train)

$$ \text{Traitement des données de test} $$ 

In [129]:
# Conversion des données de test en tensor
X_test = torch.FloatTensor(testing_feature_values)
y_test = torch.FloatTensor(y_test.values)

$$\text{Définition de notre réseau de neurones} $$

In [130]:
input_dim = 10
hidden_dim_1 = 20
hidden_dim_2 = 20
hidden_dim_3 = 20
output_dim = 1

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim_1, hidden_dim_2, hidden_dim_3, output_dim):
        super(NeuralNetwork, self).__init__()
        self.layer_1 = nn.Linear(input_dim, hidden_dim_1)
        self.layer_2 = nn.Linear(hidden_dim_1, hidden_dim_2)
        self.layer_3 = nn.Linear(hidden_dim_2, hidden_dim_3)
        self.layer_4 = nn.Linear(hidden_dim_3, output_dim)
       
    def forward(self, x):
        # Choix de la sigmoide car elle est plus utilisée dans les problèmes de classification
        x = torch.nn.functional.relu(self.layer_1(x))
        x = torch.nn.functional.relu(self.layer_2(x))
        x = torch.nn.functional.relu(self.layer_3(x))
        x = torch.nn.functional.sigmoid(self.layer_4(x))
        return x

In [131]:
# Déclaration du modèle
model = NeuralNetwork(input_dim, hidden_dim_1, hidden_dim_2, hidden_dim_3, output_dim)

In [132]:
model.parameters

<bound method Module.parameters of NeuralNetwork(
  (layer_1): Linear(in_features=10, out_features=20, bias=True)
  (layer_2): Linear(in_features=20, out_features=20, bias=True)
  (layer_3): Linear(in_features=20, out_features=20, bias=True)
  (layer_4): Linear(in_features=20, out_features=1, bias=True)
)>

In [133]:
# Set the criterion of the model to measure error
criterion = nn.BCELoss()
# Choose our Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.1)

In [134]:
num_epochs = 100
loss_values = []


for epoch in range(num_epochs):
    model.train()
    y_pred = model.forward(X_train)
    loss = criterion(y_pred, y_train)
    loss_values.append(loss.detach().numpy())

    if epoch%10 == 0 :
        print(f'Epoch : {epoch} and loss :{loss}')

    # Back propagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print("Training Complete")


Epoch : 0 and loss :0.7378884553909302
Epoch : 10 and loss :0.30384713411331177
Epoch : 20 and loss :0.2751396596431732
Epoch : 30 and loss :0.27465492486953735
Epoch : 40 and loss :0.2725161015987396
Epoch : 50 and loss :0.27093636989593506
Epoch : 60 and loss :0.27018773555755615
Epoch : 70 and loss :0.269695907831192
Epoch : 80 and loss :0.26933762431144714
Epoch : 90 and loss :0.26893237233161926
Training Complete


In [135]:
model.eval()

with torch.no_grad():
    predictions = model(X_test)

y_pred = (predictions > 0.5).float()

In [136]:
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, f1_score, confusion_matrix

In [137]:
accuracy_score(y_test, y_pred)
confusion_matrix(y_test, y_pred)

array([[48547,     0],
       [ 4879,     0]], dtype=int64)

In [138]:
from sklearn.metrics import roc_auc_score, f1_score, recall_score, accuracy_score

In [139]:
print("Roc_Auc : ",roc_auc_score(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test,y_pred))
print("Recall_score : ",recall_score(y_test, y_pred))
print("F1_score :", f1_score(y_test, y_pred))

Roc_Auc :  0.5
Accuracy:  0.9086774229775765
Recall_score :  0.0
F1_score : 0.0
